# Severity Exploration

## Overview
In Part I, we set up a manufacturer with a loss structure (attritional, large, and catastrophic losses).
In Part II, we plot the true severity distribution and explore its shape.
In Part III, we draw a 5-year sample from the "true" distribution, and approximate it using Maximum Likelihood. The kicker is we have no significant catastrophic losses in the dataset, so we simulate an industry database of $5M+ catastrophic losses from a pool of around 10,000 companies per year (this annual number of companies is Poisson-distributed) and explore the catastrophic tail shape from that data.

- **Audience**: [Practitioner] / [Developer]

In [1]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/optimization'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install ergodic-insurance -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Setup complete. If you see numpy/scipy import errors below,
restart the runtime (Runtime > Restart runtime) and re-run all cells.


## Setup

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import multiprocessing

warnings.filterwarnings("ignore")

from ergodic_insurance.pareto_frontier import (
    Objective, ObjectiveType, ParetoFrontier, ParetoPoint,
)
from ergodic_insurance.config import ManufacturerConfig
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.insurance_program import (
    EnhancedInsuranceLayer, InsuranceProgram,
)
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator

plt.style.use("seaborn-v0_8-darkgrid")
SEED = 42
np.random.seed(SEED)
N_CORES = multiprocessing.cpu_count()
print(f"Number of CPU cores: {N_CORES}")  # Available parallel cores for sensitivity sweep
CI = False      # Set True to skip heavy computations

Number of CPU cores: 44


## Part I: Parameter Setup

### Manufacturing Company Configuration

Baseline company parameters (the experiment varies revenue via `revenue_grid`).

In [ ]:
# --- Economic Parameters
ATR = 2.0                # Asset turnover ratio
OPERATING_MARGIN = 0.15  # 12% EBIT margin before Insurable Losses
REV_VOL = 0.50           # Revenue volatility (annualized)
INITIAL_ASSETS = 10_000_000

# --- Company Configuration ---
mfg_config = ManufacturerConfig(
    initial_assets=INITIAL_ASSETS,          # $15M total assets
    asset_turnover_ratio=ATR,               # Revenue = Assets Ãƒâ€” turnover = $22.5M
    base_operating_margin=OPERATING_MARGIN, # 12% EBIT margin -> $2.7M/yr operating income
    tax_rate=0.25,                          # 25% corporate tax
    retention_ratio=0.70,                   # 70% earnings retained for growth
)

# Display company profile
revenue = mfg_config.initial_assets * mfg_config.asset_turnover_ratio
ebit = revenue * mfg_config.base_operating_margin
print("=" * 60)
print("MANUFACTURING COMPANY PROFILE")
print("=" * 60)
print(f"Total Assets:          ${mfg_config.initial_assets:>14,.0f}")
print(f"Annual Revenue:        ${revenue:>14,.0f}")
print(f"Operating Income:      ${ebit:>14,.0f}")
print(f"Operating Margin:      {mfg_config.base_operating_margin:>14.1%}")
print(f"Asset Turnover:        {mfg_config.asset_turnover_ratio:>14.1f}x")
print(f"Revenue Volatility:    {REV_VOL:>14}")
print(f"Tax Rate:              {mfg_config.tax_rate:>13.1%}")
print(f"Retention Ratio:       {mfg_config.retention_ratio:>13.1%}")
print("=" * 60)

MANUFACTURING COMPANY PROFILE
Total Assets:          $     5,000,000
Annual Revenue:        $    10,000,000
Operating Income:      $     1,500,000
Operating Margin:               15.0%
Asset Turnover:                   2.0x
Revenue Volatility:               0.5
Tax Rate:                      25.0%
Retention Ratio:               70.0%


### Shared Simulation Infrastructure

Loss model, analytical LEV-based layer pricing, CRN scenario generation, and simulation engine.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
from concurrent.futures import ProcessPoolExecutor
import warnings
import logging
import time

# Suppress all warnings and verbose solver logging
warnings.filterwarnings("ignore")
logging.getLogger("ergodic_insurance").setLevel(logging.ERROR)

from ergodic_insurance.hjb_solver import (
    StateVariable, ControlVariable, StateSpace,
    LogUtility, PowerUtility, ExpectedWealth,
    HJBProblem, HJBSolver, HJBSolverConfig,
)
from ergodic_insurance.optimal_control import (
    ControlSpace, StaticControl, HJBFeedbackControl,
    TimeVaryingControl, OptimalController,
)
from ergodic_insurance.config import ManufacturerConfig
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.insurance_program import (
    EnhancedInsuranceLayer, InsuranceProgram,
)
from ergodic_insurance.loss_distributions import (
    ManufacturingLossGenerator, LognormalLoss, ParetoLoss,
)
from ergodic_insurance.insurance_pricing import LayerPricer

plt.style.use("seaborn-v0_8-darkgrid")
SEED = 42
np.random.seed(SEED)
N_CORES = 40   # Available parallel cores for sensitivity sweep
CI = False      # Set True to skip heavy computations

# =====================================================
# SHARED SIMULATION INFRASTRUCTURE
# =====================================================
# Used by Parts 5, 8, 9, 10, and 11.

# --- Economic Parameters ---
REFERENCE_REVENUE = ATR * INITIAL_ASSETS  # Fixed reference for loss calibration

# --- Loss Scaling ---
# Loss frequency (and CRN loss amounts) scale with the square root of
# revenue.  This keeps the loss drag proportional to the company's
# actual size for both insured and uninsured strategies.
FREQ_SCALING_EXPONENT = 0.75

# --- Amplified Loss Model ---
ATTR_BASE_FREQ = 3
ATTR_SEV_MEAN = 10_000
ATTR_SEV_CV = 15

LG_BASE_FREQ = 1.25
LG_SEV_MEAN = 400_000
LG_SEV_CV = 7

CAT_BASE_FREQ = 0.15
CAT_SEV_ALPHA = 2.01
CAT_SEV_XM = 1_000_000

LOSS_PARAMS = dict(
    attritional_params={'base_frequency': ATTR_BASE_FREQ,
                        'severity_mean': ATTR_SEV_MEAN,
                        'severity_cv': ATTR_SEV_CV,
                        'revenue_scaling_exponent': FREQ_SCALING_EXPONENT,
                        'reference_revenue': REFERENCE_REVENUE},
    large_params={'base_frequency': LG_BASE_FREQ,
                  'severity_mean': LG_SEV_MEAN,
                  'severity_cv': LG_SEV_CV,
                  'revenue_scaling_exponent': FREQ_SCALING_EXPONENT,
                  'reference_revenue': REFERENCE_REVENUE},
    catastrophic_params={'base_frequency': CAT_BASE_FREQ,
                         'severity_alpha': CAT_SEV_ALPHA,
                         'severity_xm': CAT_SEV_XM,
                         'revenue_scaling_exponent': FREQ_SCALING_EXPONENT,
                         'reference_revenue': REFERENCE_REVENUE},
)

# Quick validation of the loss model
_val_gen = ManufacturingLossGenerator(**LOSS_PARAMS, seed=99)
_val_totals = []
SCENARIOS = 10_000
for _ in range(SCENARIOS):
    _events, _stats = _val_gen.generate_losses(duration=1.0, revenue=REFERENCE_REVENUE)
    _val_totals.append(_stats['total_amount'])
_expected_annual_loss = np.mean(_val_totals)
_operating_income = INITIAL_ASSETS * ATR * OPERATING_MARGIN
print(f"Loss model validation ({SCENARIOS:,.0f} one-year samples):")
print(f"  Expected annual loss:  ${_expected_annual_loss:>12,.0f}")
print(f"  Operating income:      ${_operating_income:>12,.0f}")
print(f"  Loss / Income ratio:   {_expected_annual_loss / _operating_income:.0%}")
print(f"  Std dev annual loss:   ${np.std(_val_totals):>12,.0f}")
print(f"  Max annual loss:       ${np.max(_val_totals):>12,.0f}")
del _val_gen, _val_totals, _events, _stats


# --- Analytical Layer Pricing via LEV ---
# Instead of hardcoded rate-on-line values, we compute actuarially sound
# premiums from the known severity distributions using limited expected
# values (LEVs).  For each layer (attachment a, limit l):
#
#   E[layer loss] = sum_i  freq_i * [LEV_i(a+l) - LEV_i(a)]
#   premium       = E[layer loss] / target_loss_ratio
#   rate_on_line  = premium / limit
#
# This ensures the primary-layer ROL decreases naturally as the Ded
# (retention) rises, producing the genuine cost-vs-variance tradeoff
# that the HJB solver needs.
#
# The pricers are parameterized so that the sensitivity analysis (Part 9)
# can adapt premiums to match the modified loss assumptions being tested.

TARGET_LOSS_RATIO = 0.7  # Normal-market loss ratio
LOSS_RATIO_INFLECTION = 1  # Factor by which the max layer loss ratio differs from the base

def make_layer_pricers(large_freq=LG_BASE_FREQ,
                        large_sev_mean=LG_SEV_MEAN,
                        cur_revenue=REFERENCE_REVENUE) -> tuple:
    """Create a tuple of LayerPricers for a given loss parameterization.

    Frequency scales as (revenue / reference)^0.5, matching the loss
    model's sub-linear revenue scaling.  This keeps premium and loss
    scaling consistent so that insured and uninsured strategies face
    the same proportional cost growth.

    Args:
        large_freq: Large-loss annual frequency (default 1.0).
        large_sev_mean: Large-loss mean severity (default $1M).
        cur_revenue: Current revenue for frequency scaling.

    Returns:
        Tuple of (attritional, large, catastrophic) LayerPricers.
    """
    scale = (cur_revenue / REFERENCE_REVENUE) ** FREQ_SCALING_EXPONENT
    return (
        LayerPricer(LognormalLoss(mean=ATTR_SEV_MEAN, cv=ATTR_SEV_CV),
                    frequency=ATTR_BASE_FREQ * scale),
        LayerPricer(LognormalLoss(mean=large_sev_mean, cv=LG_SEV_CV),
                    frequency=large_freq * scale),
        LayerPricer(ParetoLoss(alpha=CAT_SEV_ALPHA, xm=CAT_SEV_XM),
                    frequency=CAT_BASE_FREQ * scale),
    )


# Default pricers for baseline loss model
DEFAULT_PRICERS = make_layer_pricers()

MIN_LAYER_MIDPOINT = np.mean([0, 5_000_000])
MAX_LAYER_MIDPOINT = np.mean([450_000_000, 500_000_000])

def analytical_layer_premium(attachment: float, limit: float,
                            base_loss_ratio: float,
                            loss_ratio_inflection: float,
                            pricers=None) -> float:
    """Compute actuarial premium for a layer using LEV-based expected losses.

    Premium = E[layer loss] / layer_loss_ratio, where:
      E[layer loss] = sum over components of freq_i * (LEV_i(a+l) - LEV_i(a))

    Args:
        attachment: Layer attachment point.
        limit: Layer limit (width of coverage).
        pricers: Tuple of LayerPricers. Uses DEFAULT_PRICERS if None.
        base_loss_ratio: Desired loss ratio for the layer.
        loss_ratio_inflection: Factor by which the max layer loss ratio  differs from the base.
    """

    pricers = pricers or DEFAULT_PRICERS
    expected_loss = sum(p.expected_layer_loss(attachment, limit) for p in pricers)
    cur_layer_midpoint = np.mean([attachment, attachment + limit])
    layer_loss_ratio = base_loss_ratio + \
                        (1.0 / loss_ratio_inflection - 1.0) * base_loss_ratio \
                        * (cur_layer_midpoint - MIN_LAYER_MIDPOINT) \
                        / (MAX_LAYER_MIDPOINT - MIN_LAYER_MIDPOINT)
    return expected_loss / layer_loss_ratio


def analytical_rate_on_line(attachment: float, limit: float,
                            base_loss_ratio: float,
                            loss_ratio_inflection: float,
                            pricers=None) -> float:
    """Compute rate-on-line for a layer: premium / limit."""
    if limit <= 0:
        return 0.0
    return analytical_layer_premium(attachment,
                                    limit,
                                    base_loss_ratio,
                                    loss_ratio_inflection,
                                    pricers) / limit


# Validate: show how ROL varies across sample attachment points
print(f"\nAnalytical layer pricing (target LR = {TARGET_LOSS_RATIO:.0%}):")
print(f"  {'Attachment':>12s}  {'Limit':>12s}  {'E[Loss]':>12s}  {'Premium':>12s}  {'ROL':>8s}")
print(f"  {'-'*12}  {'-'*12}  {'-'*12}  {'-'*12}  {'-'*8}")
for _a, _l in [(10_000, 4_990_000), (25_000, 4_975_000), (50_000, 4_950_000),
                (250_000, 4_750_000), (1_000_000, 4_000_000),
                (2_000_000, 3_000_000), (4_000_000, 1_000_000),
                (5_000_000, 20_000_000), (25_000_000, 25_000_000), (50_000_000, 50_000_000)]:
    _el = analytical_layer_premium(_a, _l, TARGET_LOSS_RATIO, 1.0) * TARGET_LOSS_RATIO
    _p = analytical_layer_premium(_a, _l, TARGET_LOSS_RATIO, 1.0)
    _r = analytical_rate_on_line(_a, _l, TARGET_LOSS_RATIO, 1.0)
    print(f"  ${_a:>11,.0f}  ${_l:>11,.0f}  ${_el:>11,.0f}  ${_p:>11,.0f}  {_r:>7.2%}")


# --- Insurance Tower Factory ---
# Premium rates are computed analytically from the loss distribution,
# ensuring that the primary-layer ROL decreases with higher retention.
# The optional `pricers` argument lets the sensitivity analysis pass
# in LayerPricers built from alternative loss assumptions, so that
# premiums stay consistent with the loss environment being tested.

def make_program(ded: float,
                base_loss_ratio: float,
                loss_ratio_inflection: float,
                max_limit: float,
                pricers=None) -> InsuranceProgram:
    """Create 4-layer tower with analytically priced premiums.

    Uses LEV-based layer pricing from severity distributions so that
    rate-on-line adjusts naturally with the retention level.

    Args:
        ded: Deductible.
        max_limit: Maximum coverage limit (top of tower).
        pricers: Tuple of LayerPricers. Uses DEFAULT_PRICERS if None.

    Returns:
        InsuranceProgram with actuarially sound premium loading.
    """
    layer_defs = [
        # (attachment, ceiling, reinstatements)
        (0, 5_000_000, 0),
        (5_000_000, 10_000_000, 0),
        (10_000_000, 25_000_000, 0),
        (25_000_000, 50_000_000, 0),
        (50_000_000, 100_000_000, 0),
        (100_000_000, 150_000_000, 0),
        (150_000_000, 200_000_000, 0),
        (200_000_000, 250_000_000, 0),
        (250_000_000, 300_000_000, 0),
        (300_000_000, 350_000_000, 0),
        (350_000_000, 400_000_000, 0),
        (400_000_000, 450_000_000, 0),
        (450_000_000, 500_000_000, 0),
    ]
    layers = []
    for attach, ceiling, reinst in layer_defs:
        if ded >= ceiling:
            continue  # Skip layers that are fully below the deductible
        if max_limit is not None and ceiling > max_limit:
            continue  # Skip layers that exceed the max limit constraint
        # Now we're within the working layer
        # The deductible is below, the max limit is above
        effective_attach = max(attach, ded)
        limit = ceiling - effective_attach
        if limit <= 0:
            continue
        rol = analytical_rate_on_line(effective_attach, limit, base_loss_ratio, loss_ratio_inflection, pricers)
        layers.append(EnhancedInsuranceLayer(
            attachment_point=effective_attach,
            limit=limit,
            base_premium_rate=rol,
            reinstatements=reinst,
        ))
    return InsuranceProgram(
        layers=layers,
        deductible=ded,
        name=f"Manufacturing Tower (Ded=${ded:,.0f})",
    )


# --- CRN: Pre-generate Loss Scenarios ---
def generate_loss_pool(n_paths, n_years, reference_revenue=REFERENCE_REVENUE, seed=SEED, specific_loss_params=None):
    """Pre-generate loss scenarios for Common Random Number comparison.
    All strategies will face the exact same loss events and revenue shocks.
    Losses are generated at a fixed reference revenue; the simulation
    engine then scales event amounts by (actual_revenue / reference)^0.5
    so that loss burden grows proportionally with the company.

    Args:
        n_paths: Number of simulation paths.
        n_years: Number of years to simulate.
        reference_revenue: Reference revenue for loss calibration.
        seed: Base seed for random number generation.
        specific_loss_params: A dictionary of loss parameters to use, overriding
                              the global LOSS_PARAMS for this call.
    """
    loss_params_to_use = specific_loss_params if specific_loss_params is not None else LOSS_PARAMS

    ss = np.random.SeedSequence(seed)
    children = ss.spawn(n_paths + 1)

    # Shared revenue shocks
    rev_rng = np.random.default_rng(children[0])
    revenue_shocks = rev_rng.standard_normal((n_paths, n_years))

    # Per-path loss event sequences
    all_losses = []  # [path][year] -> List[LossEvent]
    for i in range(n_paths):
        gen = ManufacturingLossGenerator(
            **loss_params_to_use, # Use the potentially overridden loss params
            seed=int(children[i + 1].generate_state(1)[0] % (2**31)),
        )
        path_losses = []
        for t in range(n_years):
            events, _ = gen.generate_losses(duration=1.0, revenue=reference_revenue)
            path_losses.append(events)
        all_losses.append(path_losses)

    return revenue_shocks, all_losses


# --- CRN Simulation Engine ---
def simulate_with_crn(ded,
                    base_loss_ratio: float,
                    loss_ratio_inflection: float,
                    max_limit: float,
                    revenue_shocks, loss_pool, n_years=1,
                    initial_assets=INITIAL_ASSETS, pricers=None):
    """Simulate one static-Ded strategy across all CRN paths.

    Uses the library's InsuranceProgram.process_claim() to correctly
    allocate each loss through the insurance tower.

    Loss amounts from the CRN pool are scaled by
    (actual_revenue / REFERENCE_REVENUE)^FREQ_SCALING_EXPONENT so that
    the loss burden grows proportionally with the company.  Premium is
    repriced at actual revenue with the same exponent, keeping the
    cost-of-risk consistent between insured and uninsured strategies.

    Args:
        ded: Deductible.
        revenue_shocks: Pre-generated revenue shocks (n_paths x n_years).
        loss_pool: Pre-generated loss events [path][year] -> List[LossEvent].
        n_years: Simulation horizon.
        initial_assets: Starting wealth.
        pricers: Tuple of LayerPricers for premium calculation.
            Uses DEFAULT_PRICERS if None (baseline loss assumptions).

    Returns:
        paths: array of shape (n_paths, n_years + 1) with asset values.
    """
    n_paths = len(loss_pool)
    paths = np.zeros((n_paths, n_years + 1))
    paths[:, 0] = initial_assets

    # Build program template and get fixed annual premium
    if ded >= 100_000_000:
        # "No insurance" -- skip tower entirely
        annual_premium = 0.0
        use_insurance = False
        program_template = None
    else:
        program_template = make_program(ded,
                                        base_loss_ratio,
                                        loss_ratio_inflection,
                                        max_limit,
                                        pricers=pricers)
        annual_premium = program_template.calculate_premium()
        use_insurance = True

    for i in range(n_paths):
        assets = initial_assets
        for t in range(n_years):
            # Operating income with shared revenue shock
            revenue = assets * ATR * np.exp(
                REV_VOL * revenue_shocks[i, t] - 0.5 * REV_VOL**2
            )
            operating_income = revenue * OPERATING_MARGIN

            # Scale CRN losses to current revenue (sqrt scaling)
            loss_scale = (revenue / REFERENCE_REVENUE) ** FREQ_SCALING_EXPONENT

            # Process losses through insurance tower
            total_retained = 0.0
            if use_insurance:
                new_pricers = make_layer_pricers(cur_revenue=revenue)
                program_update = make_program(ded,
                                                base_loss_ratio,
                                                loss_ratio_inflection,
                                                max_limit,
                                                pricers=new_pricers)
                annual_premium = program_update.calculate_premium()
                program = InsuranceProgram.create_fresh(program_update)
                for event in loss_pool[i][t]:
                    scaled_amount = event.amount * loss_scale
                    result = program.process_claim(scaled_amount)
                    total_retained += result.deductible_paid + result.uncovered_loss
            else:
                for event in loss_pool[i][t]:
                    total_retained += event.amount * loss_scale

            # Net income and asset update
            assets = assets + operating_income - total_retained - annual_premium
            assets = max(assets, 0.0)
            paths[i, t + 1] = assets

    return paths


# Pre-generate the main CRN pool
N_PATHS = 500
N_YEARS = 1
print(f"\nPre-generating CRN loss pool ({N_PATHS:,.0f} paths x {N_YEARS} years)...")
t0 = time.time()
CRN_SHOCKS, CRN_LOSSES = generate_loss_pool(n_paths=N_PATHS, n_years=N_YEARS)
print(f"  Done in {time.time() - t0:.1f}s")
print(f"  Shape: {CRN_SHOCKS.shape[0]:,} paths x {CRN_SHOCKS.shape[1]} years")

# Quick sanity: total losses per path-year
_annual_totals = [
    sum(e.amount for e in CRN_LOSSES[i][t])
    for i in range(N_PATHS) for t in range(N_YEARS)
]
print(f"  Mean annual loss: ${np.mean(_annual_totals):,.0f}")
del _annual_totals


Loss model validation (10,000 one-year samples):
  Expected annual loss:  $     810,152
  Operating income:      $   1,500,000
  Loss / Income ratio:   54%
  Std dev annual loss:   $   2,232,722
  Max annual loss:       $  78,067,932

Analytical layer pricing (target LR = 70%):
    Attachment         Limit       E[Loss]       Premium       ROL
  ------------  ------------  ------------  ------------  --------
  $     10,000  $  4,990,000  $    658,191  $    940,273   18.84%
  $     25,000  $  4,975,000  $    638,549  $    912,213   18.34%
  $     50,000  $  4,950,000  $    613,275  $    876,107   17.70%
  $    250,000  $  4,750,000  $    493,355  $    704,793   14.84%
  $  1,000,000  $  4,000,000  $    260,635  $    372,335    9.31%
  $  2,000,000  $  3,000,000  $    121,171  $    173,101    5.77%
  $  4,000,000  $  1,000,000  $     24,540  $     35,057    3.51%
  $  5,000,000  $ 20,000,000  $    109,088  $    155,841    0.78%
  $ 25,000,000  $ 25,000,000  $     20,511  $     29,301   

In [5]:
# =====================================================
# SOBOL QUASI-RANDOM LOSS GENERATION & IMPORTANCE SAMPLING
# =====================================================
# Quasi-random low-discrepancy sequences for variance reduction,
# plus rejection-based importance sampling for tail events.

from scipy.stats.qmc import Sobol as _Sobol
from scipy.stats import norm as _sp_norm, poisson as _sp_poisson

# Truncation limits for compound Poisson event counts
MAX_ATTR_EVENTS  = 10
MAX_LARGE_EVENTS = 6
MAX_CAT_EVENTS   = 3
MAX_EVENTS = MAX_ATTR_EVENTS + MAX_LARGE_EVENTS + MAX_CAT_EVENTS   # 19
SOBOL_DIMS = 1 + (1 + MAX_ATTR_EVENTS) + (1 + MAX_LARGE_EVENTS) + (1 + MAX_CAT_EVENTS)  # 23


def generate_sobol_loss_pool(n_paths, revenue, seed=0):
    """Generate loss scenarios using 23-dim Sobol quasi-random sequences.

    Dimension layout:
      0:       revenue shock  ~ N(0,1)
      1-11:    attritional count (Poisson) + 10 severities (Lognormal)
      12-18:   large count (Poisson) + 6 severities (Lognormal)
      19-22:   cat count (Poisson) + 3 severities (Pareto)

    Returns:
        revenue_shocks:       (n_paths, 1) standard normal shocks
        loss_amounts:         (n_paths, MAX_EVENTS) individual losses (0 = unused slot)
        max_individual_loss:  (n_paths,) largest single loss per path
    """
    scale = (revenue / REFERENCE_REVENUE) ** FREQ_SCALING_EXPONENT
    attr_freq = ATTR_BASE_FREQ * scale
    lg_freq   = LG_BASE_FREQ * scale
    cat_freq  = CAT_BASE_FREQ * scale

    attr_var = np.log(1 + ATTR_SEV_CV**2)
    attr_mu  = np.log(ATTR_SEV_MEAN) - attr_var / 2
    attr_sig = np.sqrt(attr_var)

    lg_var = np.log(1 + LG_SEV_CV**2)
    lg_mu  = np.log(LG_SEV_MEAN) - lg_var / 2
    lg_sig = np.sqrt(lg_var)

    sampler = _Sobol(d=SOBOL_DIMS, scramble=True, seed=seed)
    m = int(np.ceil(np.log2(max(n_paths, 2))))
    U = sampler.random(2**m)[:n_paths]
    eps = 1e-10
    col = 0

    # Revenue shock ~ N(0,1)
    revenue_shocks = _sp_norm.ppf(np.clip(U[:, col], eps, 1 - eps)).reshape(-1, 1)
    col += 1

    # Attritional: Poisson count + Lognormal severities
    attr_n = _sp_poisson.ppf(np.clip(U[:, col], 0, 1 - eps), attr_freq).astype(int)
    attr_n = np.minimum(attr_n, MAX_ATTR_EVENTS)
    col += 1
    attr_raw = np.exp(attr_mu + attr_sig * _sp_norm.ppf(
        np.clip(U[:, col:col + MAX_ATTR_EVENTS], eps, 1 - eps)))
    col += MAX_ATTR_EVENTS
    attr_sevs = attr_raw * (np.arange(MAX_ATTR_EVENTS) < attr_n[:, None])

    # Large: Poisson count + Lognormal severities
    lg_n = _sp_poisson.ppf(np.clip(U[:, col], 0, 1 - eps), lg_freq).astype(int)
    lg_n = np.minimum(lg_n, MAX_LARGE_EVENTS)
    col += 1
    lg_raw = np.exp(lg_mu + lg_sig * _sp_norm.ppf(
        np.clip(U[:, col:col + MAX_LARGE_EVENTS], eps, 1 - eps)))
    col += MAX_LARGE_EVENTS
    lg_sevs = lg_raw * (np.arange(MAX_LARGE_EVENTS) < lg_n[:, None])

    # Catastrophic: Poisson count + Pareto severities
    cat_n = _sp_poisson.ppf(np.clip(U[:, col], 0, 1 - eps), cat_freq).astype(int)
    cat_n = np.minimum(cat_n, MAX_CAT_EVENTS)
    col += 1
    cat_raw = CAT_SEV_XM / (1 - np.clip(
        U[:, col:col + MAX_CAT_EVENTS], 0, 1 - eps)) ** (1 / CAT_SEV_ALPHA)
    col += MAX_CAT_EVENTS
    cat_sevs = cat_raw * (np.arange(MAX_CAT_EVENTS) < cat_n[:, None])

    loss_amounts = np.hstack([attr_sevs, lg_sevs, cat_sevs])
    max_individual_loss = np.max(loss_amounts, axis=1)

    return revenue_shocks, loss_amounts, max_individual_loss


def generate_is_pool_rejection(n_target, revenue, threshold=50_000_000,
                               seed_offset=1_000_000, batch_size=260_000,
                               verbose=False):
    """Generate importance-sampled tail paths via rejection sampling.

    Generates Sobol batches, keeps only paths where the max individual
    loss exceeds the threshold.  Continues until n_target paths collected.

    Returns:
        is_shocks:  (n_target, 1) revenue shocks for tail paths
        is_losses:  (n_target, MAX_EVENTS) loss amounts for tail paths
        p_tail:     estimated tail probability (acceptance rate)
    """
    collected_shocks, collected_losses = [], []
    n_generated = n_accepted = batch_num = 0

    while n_accepted < n_target:
        shocks, losses, max_loss = generate_sobol_loss_pool(
            batch_size, revenue, seed=seed_offset + batch_num)
        mask = max_loss > threshold
        n_hit = int(mask.sum())
        if n_hit > 0:
            collected_shocks.append(shocks[mask])
            collected_losses.append(losses[mask])
            n_accepted += n_hit
        n_generated += len(max_loss)
        batch_num += 1
        if verbose and batch_num % 10 == 0:
            print(f"  IS rejection: {n_accepted}/{n_target} from "
                  f"{n_generated:,} ({n_accepted/max(n_generated,1):.6%})")

    is_shocks = np.vstack(collected_shocks)[:n_target]
    is_losses = np.vstack(collected_losses)[:n_target]
    p_tail = n_accepted / n_generated

    if verbose:
        print(f"  IS complete: {n_target} tail paths from {n_generated:,} "
              f"({p_tail:.6%}, {batch_num} batches)")

    return is_shocks, is_losses, p_tail


def compute_stratified_stats(W_T_main, W_T_is, tail_mask_main, initial_assets):
    """Combine main and IS pools via stratified estimation.

    The tail probability is estimated from the main pool (unbiased).
    Tail-stratum statistics use both main-pool tail paths and IS paths
    for improved precision in the extreme-loss regime.

    Returns:
        (growth_rate, growth_vol, ruin_prob)
    """
    N_main = len(W_T_main)
    p_tail = tail_mask_main.sum() / N_main

    log_g_main = np.log(np.maximum(W_T_main, 1.0) / initial_assets)
    log_g_is   = np.log(np.maximum(W_T_is, 1.0) / initial_assets)

    normal_mask = ~tail_mask_main
    if normal_mask.any():
        lg_n = log_g_main[normal_mask]
        mean_n, meansq_n = lg_n.mean(), (lg_n**2).mean()
        ruin_n = float((W_T_main[normal_mask] <= 0).mean())
    else:
        mean_n = meansq_n = ruin_n = 0.0

    # Tail stratum: main-pool tail paths + IS paths
    if tail_mask_main.any():
        lg_t = np.concatenate([log_g_main[tail_mask_main], log_g_is])
        wt_t = np.concatenate([W_T_main[tail_mask_main], W_T_is])
    else:
        lg_t, wt_t = log_g_is, W_T_is

    mean_t, meansq_t = lg_t.mean(), (lg_t**2).mean()
    ruin_t = float((wt_t <= 0).mean())

    growth_rate = (1 - p_tail) * mean_n + p_tail * mean_t
    E_sq = (1 - p_tail) * meansq_n + p_tail * meansq_t
    growth_vol  = np.sqrt(max(E_sq - growth_rate**2, 0.0))
    ruin_prob   = (1 - p_tail) * ruin_n + p_tail * ruin_t

    return growth_rate, growth_vol, ruin_prob


# Quick validation of the Sobol generator
_val_shocks, _val_losses, _val_max = generate_sobol_loss_pool(10_000, REFERENCE_REVENUE, seed=99)
_val_totals = _val_losses.sum(axis=1)
print(f"\nSobol loss pool validation (10,000 paths at reference revenue):")
print(f"  Mean annual loss:   ${np.mean(_val_totals):>12,.0f}")
print(f"  Std dev:            ${np.std(_val_totals):>12,.0f}")
print(f"  Max single event:   ${np.max(_val_max):>12,.0f}")
print(f"  Paths with loss > $50M event: {(np.max(_val_losses, axis=1) > 50_000_000).sum()}")
print(f"  Sobol dims: {SOBOL_DIMS}, Max events/path: {MAX_EVENTS}")
del _val_shocks, _val_losses, _val_max, _val_totals


Sobol loss pool validation (10,000 paths at reference revenue):
  Mean annual loss:   $     835,429
  Std dev:            $   2,902,205
  Max single event:   $ 141,541,691
  Paths with loss > $50M event: 4
  Sobol dims: 23, Max events/path: 19


## Part II: Plotting the Severity Distribution

## Part III: Estimating Attritional and Large Loss Frequency and Severity with Shadow Moments

## Part IV: Estimating the Tail Shape from Industry Data

## Part V: Comparing the "True" Distribution with MLE Estimate